In [1]:
import pandas as pd
import numpy as np

##### 13~16

In [39]:
import pandas as pd
import numpy as np

def experiment(sample_size, tau):
    # Generate random array x and compute y
    x = np.random.uniform(-0.5, 0.5, sample_size)
    x.sort()
    y = np.sign(x)

    def flip_labels(y, tau):
        if tau > 0:
            random_vals = np.random.random(size=y.shape)
            flip_indices = random_vals <= tau
            y[flip_indices] *= -1
        return y

    # Flip y to -y with probability tau
    y = flip_labels(y, tau)

    # Generate new random array x and compute y
    x_new = np.random.uniform(-0.5, 0.5, sample_size)
    x_new.sort()
    y_new = np.sign(x_new)

    y_new = flip_labels(y_new, tau)

    # Define the hypothesis h
    def h(x, s, theta):
        return s * np.sign(x - theta)

    # Compute possible values of theta
    diffs = np.diff(x)
    if len(diffs) == 0:
        theta_vals = [x[0]]
    else:
        theta_vals = np.hstack((x.min() - 0.1, (x[:-1] + x[1:]) / 2, x.max() + 0.1))

    # Compute Ein(h) for all combinations of s and theta
    misclassified = []
    for s in [-1, 1]:
        for theta in theta_vals:
            predictions = h(x, s, theta)
            misclassified.append((predictions != y).sum())

    # Compute the minimum misclassification error and corresponding s and theta
    min_error_idx = np.argmin(misclassified)
    min_error = misclassified[min_error_idx]
    min_error_s = [-1, 1][min_error_idx // len(theta_vals)]
    min_error_theta = theta_vals[min_error_idx % len(theta_vals)]

    # Return the hypothesis with the minimum Ein as g; break ties by smallest s * theta
    g = h(x, min_error_s, min_error_theta)
    for s in [-1, 1]:
        for theta in theta_vals:
            if s == min_error_s and theta == min_error_theta:
                continue
            error_idx = np.where((s == np.array([-1, 1])))[0][0] * len(theta_vals) + np.abs(theta_vals - theta).argmin()
            if misclassified[error_idx] == min_error and s * theta < min_error_s * min_error_theta:
                g = h(x, s, theta)
                min_error_s = s
                min_error_theta = theta

    Ein_g = min_error / len(x)
    Eout_g = np.mean(h(x_new, min_error_s, min_error_theta) != y_new)
    return Eout_g - Ein_g

n_trials = 10000
sample_size = 128
tau = 0.2   # Set probability of flipping
results = [experiment(sample_size, tau) for i in range(n_trials)]
mean_diff = np.mean(results)

print("Mean of Eout(g, tau) - Ein(g) over", n_trials, "trials:", mean_diff)


Mean of Eout(g, tau) - Ein(g) over 10000 trials: 0.014203125


##### 17~20

In [ ]:
df = pd.read_csv('ML_HW2_data_train.dat', sep="\t", header=None)